In [1]:
import numpy as np

# 假设有 3 台机器和 3 个操作
m = 2  # 机器数量
N = 2  # 表示机器的二进制 (0, 1)
n = 4  # 操作数量
l = 1  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[1, 2], [2, 1], [1, 2], [2, 1]])

b = np.array([[0, 0], [1, 0]])
# 非法数组(用于表示机器数)(1, 1)
bw = np.array([[1, 1], [0, 1]])
# 常数
K1 = 100
P = 30
# print(len(bw))
# 量子比特数量
num_qubits = n * N + l

In [3]:
# def pauli_z_term(index, num_qubits):
#     """初始化一个 Pauli-I 门的列表,指定索引 index 替换为 Pauli-Z 门"""
#     paulis = ['I'] * num_qubits
#     paulis[index] = 'Z'
#     return ''.join(paulis)  # 将列表转化为字符串
def pauli_z_term(indices, num_qubits):
    """生成 Pauli-Z 项"""
    z_str = ['I'] * num_qubits
    for idx in indices:
        z_str[idx] = 'Z'
    return ''.join(z_str)


def pauli_x_term(index, num_qubits):
    paulis = ['I'] * num_qubits
    paulis[index] = 'X'
    return ''.join(paulis)

In [4]:
def append_multi_rzz_term(qc, qubits, gamma):
    """
    构建一个类RZZ门，n-1个控制位，1个目标位，进行旋转操作。
    参数:
    control_qubits: List[int]，控制量子比特的索引
    target_qubit: int，目标量子比特的索引
    """
    if len(qubits) == 1:
        qc.rz(gamma, qubits[0])
    else:
        control_qubits = qubits[:N - 1]

        target_qubit = qubits[N - 1:]

        n_controls = len(control_qubits)
        if n_controls == 1:
            qc.cx(control_qubits[0], target_qubit[0])
        else:
            qc.mcx(control_qubits, target_qubit[0])
        qc.rz(gamma, target_qubit[0])
        if n_controls == 1:
            qc.cx(control_qubits[0], target_qubit[0])
        else:
            qc.mcx(control_qubits, target_qubit[0])

In [5]:
def append__zz_term(qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(gamma, qubit2)
    qc.cx(qubit1, qubit2)


def append__z_term(qc, qubit, gamma):
    qc.rz(gamma, qubit)


def append__mixer_term(qc, qubit, beta):
    qc.rx(2 * beta, qubit)

In [6]:
from qiskit.quantum_info import SparsePauliOp

# 初始化 Pauli 字符串和系数
pauli_strings = []
coefficients = []

# 构建哈密顿量
for h in range(l):
    coef = - 2 ** h / 2  # 系数为 2^h
    z_term = ['I'] * num_qubits
    z_term[n * N + h] = 'Z'  # 第 n * N + h 位是 Z，其余是 I
    pauli_strings.append(''.join(z_term))
    coefficients.append(coef)

# 转换为 SparsePauliOp 格式
hamiltonian1 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

# 输出结果
print("哈密顿量:")
print(hamiltonian1)

哈密顿量:
SparsePauliOp(['IIIIIIIIZ'],
              coeffs=[-0.5+0.j])


In [7]:
from itertools import combinations

# 初始化哈密顿量参数
pauli_strings = []
coefficients = []

# 遍历 j 和 i
for j in range(len(bw)):
    for i in range(n):
        # 枚举 k 的所有子集
        for subset_size in range(N + 1):  # 子集大小
            for subset in combinations(range(N), subset_size):
                # 计算子集 S 的系数
                coef = K1
                for k in subset:
                    coef *= (0.5 - b[j][k])  # 子集 S 中的项
                for k in range(N):
                    if k not in subset:
                        coef *= 0.5  # 非子集项的贡献

                # 生成对应的 Pauli-Z 项
                indices = [i * N + k for k in subset]  # 转为量子比特索引
                z_term = pauli_z_term(indices, num_qubits)

                # 添加到哈密顿量
                pauli_strings.append(z_term)
                coefficients.append(coef)

# 转换为 SparsePauliOp 格式
hamiltonian2 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

# 输出结果
print("哈密顿量:")
print(hamiltonian2)

哈密顿量:
SparsePauliOp(['IIIIIIIII', 'ZIIIIIIII', 'IZIIIIIII', 'ZZIIIIIII', 'IIIIIIIII', 'IIZIIIIII', 'IIIZIIIII', 'IIZZIIIII', 'IIIIIIIII', 'IIIIZIIII', 'IIIIIZIII', 'IIIIZZIII', 'IIIIIIIII', 'IIIIIIZII', 'IIIIIIIZI', 'IIIIIIZZI', 'IIIIIIIII', 'ZIIIIIIII', 'IZIIIIIII', 'ZZIIIIIII', 'IIIIIIIII', 'IIZIIIIII', 'IIIZIIIII', 'IIZZIIIII', 'IIIIIIIII', 'IIIIZIIII', 'IIIIIZIII', 'IIIIZZIII', 'IIIIIIIII', 'IIIIIIZII', 'IIIIIIIZI', 'IIIIIIZZI'],
              coeffs=[ 25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,
  25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,  25.+0.j,
  25.+0.j,  25.+0.j,  25.+0.j, -25.+0.j,  25.+0.j, -25.+0.j,  25.+0.j,
 -25.+0.j,  25.+0.j, -25.+0.j,  25.+0.j, -25.+0.j,  25.+0.j, -25.+0.j,
  25.+0.j, -25.+0.j,  25.+0.j, -25.+0.j])


In [8]:
# 初始化参数
pauli_strings = []
coefficients = []

# 第一部分：双重乘积项
for j in range(m):
    for i in range(n):
        for i_prime in range(n):
            coef = p[i, j] * p[i_prime, j] / (2 ** (2 * N))
            z_term = ['I'] * num_qubits
            for k in range(N):
                if b[j, k] == 1:
                    z_term[i * N + k] = 'Z'
                    z_term[i_prime * N + k] = 'Z'
            pauli_strings.append(''.join(z_term))
            coefficients.append(coef)

# 第二部分：结合时间和二进制位
for j in range(m):
    for i in range(n):
        for h in range(l):
            coef = -2 * p[i, j] * (2 ** (h - 1)) / (2 ** N)
            z_term = ['I'] * num_qubits
            for k in range(N):
                if b[j, k] == 1:
                    z_term[i * N + k] = 'Z'
            z_term[n * N + h] = 'Z'
            pauli_strings.append(''.join(z_term))
            coefficients.append(coef)

# 第三部分：纯二进制项
for h in range(l):
    for h_prime in range(l):
        coef = 2 ** (h - 1) * 2 ** (h_prime - 1)
        z_term = ['I'] * num_qubits
        if h == h_prime:
            z_term[n * N + h] = 'Z'
            pauli_strings.append(''.join(z_term))
            coefficients.append(coef)
        else:
            z_term1 = z_term.copy()
            z_term2 = z_term.copy()
            z_term1[n * N + h] = 'Z'
            z_term2[n * N + h_prime] = 'Z'
            pauli_strings.append(''.join(z_term1))
            coefficients.append(-coef)
            pauli_strings.append(''.join(z_term2))
            coefficients.append(-coef)
            z_term1[n * N + h_prime] = 'Z'
            pauli_strings.append(''.join(z_term1))
            coefficients.append(coef)

# 转换为 SparsePauliOp 格式
hamiltonian3 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

# 输出结果
print("哈密顿量:")
print(hamiltonian3)

哈密顿量:
SparsePauliOp(['IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'IIIIIIIII', 'ZIIIIIIII', 'ZIZIIIIII', 'ZIIIZIIII', 'ZIIIIIZII', 'ZIZIIIIII', 'IIZIIIIII', 'IIZIZIIII', 'IIZIIIZII', 'ZIIIZIIII', 'IIZIZIIII', 'IIIIZIIII', 'IIIIZIZII', 'ZIIIIIZII', 'IIZIIIZII', 'IIIIZIZII', 'IIIIIIZII', 'IIIIIIIIZ', 'IIIIIIIIZ', 'IIIIIIIIZ', 'IIIIIIIIZ', 'ZIIIIIIIZ', 'IIZIIIIIZ', 'IIIIZIIIZ', 'IIIIIIZIZ', 'IIIIIIIIZ'],
              coeffs=[ 0.0625+0.j,  0.125 +0.j,  0.0625+0.j,  0.125 +0.j,  0.125 +0.j,
  0.25  +0.j,  0.125 +0.j,  0.25  +0.j,  0.0625+0.j,  0.125 +0.j,
  0.0625+0.j,  0.125 +0.j,  0.125 +0.j,  0.25  +0.j,  0.125 +0.j,
  0.25  +0.j,  0.25  +0.j,  0.125 +0.j,  0.25  +0.j,  0.125 +0.j,
  0.125 +0.j,  0.0625+0.j,  0.125 +0.j,  0.0625+0.j,  0.25  +0.j,
  0.125 +0.j,  0.25  +0.j,  0.125 +0.j,  0.125 +0.j,  0.0625+0.j,
  0.125 +0.j,  0.0625+0.j, -

In [9]:
cost_hamiltonian = hamiltonian1 + hamiltonian2 + hamiltonian3

In [10]:
def get_cost_circuit1(gamma, qc):
    for h in range(l):
        coef = - 2 ** h
        append__z_term(qc, N * n + h, coef * gamma)
    return qc

In [11]:
import itertools


def get_cost_circuit2(gamma, qc):
    # print(len(bw))
    if len(bw) == 0:
        return qc
    all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
    for j in range(len(bw)):
        for i in range(n):
            constant = 1 / (2 ** N)  # 系数 1/2^N  
            for qubit_indices in all_combinations:  # 遍历各种组合
                qubits_index = np.array([], dtype=int)
                # 根据组合项生成子项的常数和qubit索引
                for k in range(N):
                    constant *= (1 - 2 * bw[j, k]) if k in qubit_indices else 1
                    if k in qubit_indices:
                        qubits_index = np.append(qubits_index, i * N + k)
                # print("qubits_index", qubits_index)
                append_multi_rzz_term(qc, qubits_index, 2 * K1 * constant * gamma)
    return qc

In [12]:
# import itertools
from collections import Counter


# 预生成所有比特组合
# all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
def get_cost_circuit3(gamma, qc):
    for j in range(len(b)):
        for i in range(n):  # 自身内部相乘
            all_combinations_1 = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
            # 获取 all_combinations_1 中任意两项的所有组合
            all_two_combinations = list(itertools.combinations(all_combinations_1, 2))
            for qubit_indices in all_two_combinations:  # 遍历各种组合
                # print("qubit_indices", qubit_indices)
                # sub_constant = constant
                qubits_index = np.array([], dtype=int)
                constant = 1
                # 将组合对中的每个元素（每个元素本身也是一个组合）放入数组
                for combo in qubit_indices:  # qubit_indices: ((0,), (1,))
                    for k in combo:  # combo: (0,)
                        constant = constant * (1 - 2 * b[j, k])
                        qubits_index = np.append(qubits_index, i * N + k)
                    # 将组合项中的所有元素添加到 qubits_index 中
                # 剔除出现偶次数的项，奇次数的项保留一个
                # print("qubits_index", qubits_index)
                count = Counter(qubits_index)
                # 仅保留出现次数为奇数的元素一次   
                qubits_index = [x for x in count if count[x] % 2 != 0]
                # print("qubits_index", qubits_index)
                constant = 1 / (2 ** (2 * N)) * constant * p[i, j] * p[i, j]
                append_multi_rzz_term(qc, qubits_index, 2 * P * constant * gamma)
        # 2，对应两两相乘
        all_combinations_1 = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
        all_combinations_2 = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
        for i1 in range(n - 1):
            for i2 in range(i1 + 1, n):
                for qubit_indices_1 in all_combinations_1:
                    # print("qubit_indices1", qubit_indices_1)
                    for qubit_indices_2 in all_combinations_2:
                        # print("qubit_indices2", qubit_indices_2)
                        qubits_index = np.array([], dtype=int)
                        constant = 1
                        for k1 in qubit_indices_1:  # qubit_indices: ((0,), (1,))
                            constant = constant * (1 - 2 * b[j, k1])
                            qubits_index = np.append(qubits_index, i1 * N + k1)
                        for k2 in qubit_indices_2:  # qubit_indices: ((0,), (1,))
                            constant = constant * (1 - 2 * b[j, k2])
                            qubits_index = np.append(qubits_index, i2 * N + k2)
                        constant = 1 / (2 ** (2 * N)) * constant * p[i1, j] * p[i2, j]
                        # print("qubits_index", qubits_index)
                        append_multi_rzz_term(qc, qubits_index, 2 * P * constant * gamma)
        # 对应常数相乘
        all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
        for i1 in range(n):  # 取常数 1 固定一个为 p[i_1, j]
            for i2 in range(n):  # 取 Z 的下标 Z_i2k
                for qubit_indices in all_combinations:  # 取 排列组合
                    qubits_index = np.array([], dtype=int)
                    constant = 1
                    for k in qubit_indices:
                        constant = constant * (1 - 2 * b[j, k])
                        qubits_index = np.append(qubits_index, i2 * N + k)
                    constant = 1 / (2 ** (2 * N)) * constant * p[i1, j] * p[i2, j]
                    # print("i1:",i1,"i2:",i2,)
                    # print("qubits_index", qubits_index)
                    append_multi_rzz_term(qc, qubits_index, 2 * P * constant * gamma)
        all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
        for i in range(n):
            constant = 1 / (2 ** N) * p[i, j]  # 系数

            for qubit_indices in all_combinations:  # 遍历各种组合
                constant1 = constant
                qubits_index = np.array([], dtype=int)
                # 根据组合项生成子项的常数和qubit索引
                for k in range(N):
                    constant1 *= (1 - 2 * b[j, k]) if k in qubit_indices else 1
                    if k in qubit_indices:
                        qubits_index = np.append(qubits_index, i * N + k)
                constant1 = constant1 * (sum(2 ** (h - 1) for h in range(l))) * (-2)
                append_multi_rzz_term(qc, qubits_index, 2 * P * constant1 * gamma)

            for h in range(l):
                for qubit_indices in all_combinations:  # 遍历各种组合
                    constant2 = constant
                    qubits_index = np.array([], dtype=int)
                    # 根据组合项生成子项的常数和qubit索引
                    for k in range(N):
                        constant2 *= (1 - 2 * b[j, k]) if k in qubit_indices else 1
                        if k in qubit_indices:
                            qubits_index = np.append(qubits_index, i * N + k)

                    qubits_index = np.append(qubits_index, n * N + h)
                    # print("qubits_index_hz", qubits_index)
                    constant2 = constant2 * (2 ** (h - 1)) * 2
                    append_multi_rzz_term(qc, qubits_index, 2 * P * constant2 * gamma)

            for h in range(l):
                constant3 = constant
                qubits_index = np.array([], dtype=int)
                # 根据组合项生成子项的常数和qubit索引
                qubits_index = np.append(qubits_index, n * N + h)
                # print("qubits_index_hz", qubits_index)
                constant3 = constant3 * (2 ** (h - 1)) * 2
                append_multi_rzz_term(qc, qubits_index, 2 * P * constant3 * gamma)

        for h1 in range(l):  # 遍历 h 的范围
            for h2 in range(l):  # 遍历 h' 的范围
                coef = 2 ** (h1 - 1) * 2 ** (h2 - 1)  # 计算系数权重

                # -Z_h1 项
                append__z_term(qc, n * N + h1, 2 * P * coef * gamma)

                # -Z_h2 项
                append__z_term(qc, n * N + h2, 2 * P * coef * gamma)

                # Z_h1 Z_h2 项
                if h1 != h2:  # 避免重复
                    append__zz_term(qc, n * N + h1, n * N + h2, 2 * P * coef * gamma)

In [34]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.opflow import PauliOp, StateFn
from scipy.optimize import minimize


# 1. 初始化线路
def initialize_reference_state(n_qubits):
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))
    return qc


# 2. 定义 cost Hamiltonian H_C
def build_cost_hamiltonian(n_qubits, terms):
    """
    Build a cost Hamiltonian H_C as a SparsePauliOp.
    :param n_qubits: Number of qubits
    :param terms: List of (Pauli_string, coefficient) tuples
    :return: SparsePauliOp
    """
    coeffs = []
    pauli_strings = []
    for term, coeff in terms:
        coeffs.append(coeff)
        pauli_strings.append(Pauli(term))
    return SparsePauliOp(pauli_strings, coeffs)


def generate_mixer_pool_grouped(n_qubits):
    """
    生成一个候选池 mixer_pool for ADAPT-QAOA.
    每组包含所有 qubit 的单比特操作符（X、Y）或双比特操作符（XX、YY）。

    :param n_qubits: 量子比特数
    :return: mixer_pool，每个元素是一个 Pauli 对象的列表
    """
    mixer_pool = []

    # 单比特操作符分组
    x_group = [Pauli('I' * i + 'X' + 'I' * (n_qubits - 1 - i)) for i in range(n_qubits)]
    y_group = [Pauli('I' * i + 'Y' + 'I' * (n_qubits - 1 - i)) for i in range(n_qubits)]
    mixer_pool.append(x_group)  # 单比特 X 分组
    mixer_pool.append(y_group)  # 单比特 Y 分组

    # 双比特操作符分组
    xx_group = []
    yy_group = []
    for i in range(n_qubits):
        for j in range(i + 1, n_qubits):
            xx_group.append(Pauli('I' * i + 'X' + 'I' * (j - i - 1) + 'X' + 'I' * (n_qubits - j - 1)))
            yy_group.append(Pauli('I' * i + 'Y' + 'I' * (j - i - 1) + 'Y' + 'I' * (n_qubits - j - 1)))
    mixer_pool.append(xx_group)  # 双比特 XX 分组
    mixer_pool.append(yy_group)  # 双比特 YY 分组

    return mixer_pool


# # 3. 定义 mixer pool for ADAPT-QAOA
# def generate_mixer_pool(n_qubits):
#     """
#     生成一个候选池 mixer_pool for ADAPT-QAOA.
#     包括 global, single-qubit, 和 two-qubit Pauli operators.
#     """
#     mixers = [Pauli('X' * n_qubits)]  # Global mixer: ΣX
#     for i in range(n_qubits):
#         mixers.append(Pauli('I' * i + 'X' + 'I' * (n_qubits - 1 - i)))  # Single X_i
#         mixers.append(Pauli('I' * i + 'Y' + 'I' * (n_qubits - 1 - i)))  # Single Y_i
#     for i in range(n_qubits):
#         for j in range(i + 1, n_qubits):
#             mixers.append(Pauli('I' * i + 'X' + 'I' * (j - i - 1) + 'X' + 'I' * (n_qubits - j - 1)))  # X_i X_j
#             mixers.append(Pauli('I' * i + 'Y' + 'I' * (j - i - 1) + 'Y' + 'I' * (n_qubits - j - 1)))  # Y_i Y_j
#     return mixers


# 4. 根据参数构建一个 p_layer 的量子线路
def build_qaoa_circuit(n_qubits, params, ansatz):
    """
    :param ansatz: 已选择的 mixers
    """
    qc = initialize_reference_state(n_qubits)
    for (beta, gamma), mixer in zip(params, ansatz):
        # 应用问题哈密顿量演化
        qc.compose(cost_hamiltonian_circuit(n_qubits, gamma), inplace=True)
        # 应用混合哈密顿量演化
        qc.compose(mixer_circuit(n_qubits, mixer, beta), inplace=True)
    return qc


# 构建 Cost Hamiltonian 量子线路
def cost_hamiltonian_circuit(n_qubits, gamma):
    qc = QuantumCircuit(n_qubits)
    get_cost_circuit1(gamma, qc)
    get_cost_circuit2(gamma, qc)
    get_cost_circuit3(gamma, qc)
    # for i in range(n_qubits):  # Example: a simple Z Hamiltonian
    #     qc.rz(2 * gamma, i)
    return qc


def mixer_circuit(mixer_group, beta):
    """
    根据选择的 mixer_group 构建 Mixer 量子线路。
    :param n_qubits: 量子比特数。
    :param mixer_group: 一个包含多个 Pauli 算符的组，或单个 Pauli。
    :param beta: 对应的参数 beta。
    :return: QuantumCircuit 对象。
    """
    qc = QuantumCircuit(num_qubits)
    # 如果输入是单个 Pauli，而不是组，将其转为列表处理
    if isinstance(mixer_group, Pauli):
        mixer_group = [mixer_group]
    # 遍历组内的每个 Pauli 算符
    for mixer in mixer_group:
        for i, pauli in enumerate(mixer.to_label()):
            if pauli == 'X':
                qc.rx(2 * beta, i)
            elif pauli == 'Y':
                qc.ry(2 * beta, i)
            elif pauli == 'Z':
                qc.rz(2 * beta, i)
    return qc


def merge_sparse_pauli_ops(A_list):
    """
    将一个包含多个 SparsePauliOp 的列表合并为一个 SparsePauliOp
    :param A_list: list of SparsePauliOp
    :return: SparsePauliOp
    """
    if not A_list:
        raise ValueError("A_list is empty.")

    # 合并所有 SparsePauliOp
    combined_sparse_pauli_op = A_list[0]
    for A in A_list[1:]:
        combined_sparse_pauli_op += A

    return combined_sparse_pauli_op


# 5. 为 mixer pool 的所有 mixer 计算梯度值
def compute_gradients(n_qubits, psi, H_C, mixer_pool):
    """
    :return: Gradients for all mixers
    """
    gradients = []
    for mixer in mixer_pool:
        # print(mixer)
        mixer_sparse = merge_sparse_pauli_ops(to_sparse_pauli_op(mixer))

        # print("mixer_sparse:",mixer_sparse)
        # print("H_C:",H_C)
        commutator = compute_commutator(H_C, mixer_sparse)
        pauli_terms = commutator.to_list()
        # 创建 PauliOp 对象
        pauli_ops = []
        for pauli_str, coeff in pauli_terms:
            # 将字符串表示的 Pauli 转换为 Pauli 对象
            pauli = Pauli(pauli_str)
            pauli_ops.append(PauliOp(pauli) * coeff)
        # 合并为一个总的 PauliOp
        commutator_pauli_op = sum(pauli_ops)
        # 计算算符的指数演化形式 e^(-i * H_C * Mixer)
        evolved_op = (-1j * commutator_pauli_op).exp_i()

        # 使用 StateFn 来组合 psi初始量子态和操作符
        state_fn = StateFn(psi)
        observable_fn = StateFn(evolved_op)

        # 计算期望值
        # ⟨ψ∣O∣ψ⟩ 的计算
        expectation_value = (~state_fn @ observable_fn).eval()
        # 将期望值乘以 −i 后添加到 gradients 列表
        gradients.append(-1j * expectation_value)
    return gradients


# 计算 [H_C, A] = H_C * Mixer - Mixer * H_C
def compute_commutator(H_C, A):
    """
    :return: SparsePauliOp (Commutator)
    """
    # print(A)
    # print(H_C)
    term1 = H_C.compose(A, front=False)  # H_C * A
    term2 = A.compose(H_C, front=False)  # A * H_C
    commutator = term1 - term2
    return commutator


# 6. 使用经典优化器优化参数
def optimize_parameters(cost_fn, initial_params):
    """
    :param cost_fn: 最小化函数
    :param initial_params: 随机初始参数
    :return: Optimal parameters
    """
    result = minimize(cost_fn, initial_params, method='COBYLA')
    return result.x


# # 将 mixer 转化为 SparsePauliOp
# def to_sparse_pauli_op(pauli, coeff=1.0):
#     if isinstance(pauli, SparsePauliOp):
#         return pauli
#     elif isinstance(pauli, Pauli):
#         return SparsePauliOp([pauli], [coeff])
#     else:
#         raise ValueError("Invalid mixer type, must be Pauli or SparsePauliOp.")
def to_sparse_pauli_op(pauli, coeff=1.0):
    """
    将一个或多个 Pauli 对象转为 SparsePauliOp。

    :param pauli: 单个 Pauli/SparsePauliOp，或包含这些对象的列表。
    :param coeff: 系数，默认为 1.0。
    :return: 如果输入是列表，返回对应的 SparsePauliOp 列表；否则返回单个 SparsePauliOp。
    """
    if isinstance(pauli, list):
        return [to_sparse_pauli_op(p, coeff) for p in pauli]
    elif isinstance(pauli, SparsePauliOp):
        return pauli
    elif isinstance(pauli, Pauli):
        return SparsePauliOp([pauli], [coeff])
    else:
        raise ValueError("无效的输入类型，必须为 Pauli、SparsePauliOp 或包含它们的列表。")


def run_circuit_and_get_statevector(qc):
    """
    运行一个量子电路，并返回其状态向量。
    :param qc: QuantumCircuit 对象
    :return: 状态向量 (numpy array)
    """
    backend = Aer.get_backend('statevector_simulator')
    result = execute(qc, backend).result()
    return result.get_statevector()


# Main ADAPT-QAOA function
def adapt_qaoa(n_qubits, cost_terms, max_iters=10, grad_threshold=1e-3):
    """
    :param n_qubits: Number of qubits
    :param cost_terms: List of (Pauli_string, coefficient) tuples for H_C
    :param max_iters: Maximum iterations
    :param grad_threshold: Gradient threshold for stopping
    """
    H_C = build_cost_hamiltonian(n_qubits, cost_terms)
    mixer_pool = generate_mixer_pool_grouped(n_qubits)
    psi_state = Aer.get_backend('statevector_simulator').run(
        initialize_reference_state(n_qubits)).result().get_statevector()
    params = []  # List of (beta, gamma) pairs
    ansatz = []  # Selected mixers
    print("mixer_pool:", mixer_pool)
    # Main loop
    # for _ in range(max_iters):
    gradients = compute_gradients(n_qubits, psi_state, H_C, mixer_pool)
    max_grad_idx = np.argmax(np.abs(gradients))
    # if np.abs(gradients[max_grad_idx]) < grad_threshold:
    #     break
    selected_group = mixer_pool[max_grad_idx]
    ansatz.append(selected_group)

    # Optimize variational parameters
    initial_params = np.random.rand(len(params) + 1, 2).flatten()
    # cost_fn = lambda p: -np.real(np.vdot(psi_state, build_qaoa_circuit(n_qubits, p.reshape(-1, 2), ansatz)))
    cost_fn = lambda p: -np.real(
        np.vdot(
            psi_state,  # 初始态的状态向量
            run_circuit_and_get_statevector(build_qaoa_circuit(n_qubits, p.reshape(-1, 2), ansatz))
        )
    )
    optimal_params = optimize_parameters(cost_fn, initial_params)
    params = optimal_params.reshape(-1, 2)
    return ansatz, params


In [35]:
from qiskit import Aer, execute
from qiskit.quantum_info import Pauli, SparsePauliOp
import warnings

# 忽略弃用警告
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Simple problem Hamiltonian: H_C = Z0Z1 + Z1Z2
def simple_cost_hamiltonian():
    """
    Build a simple cost Hamiltonian H_C = Z0Z1 + Z1Z2.
    :return: List of (Pauli_string, coefficient) tuples
    """
    # return [("ZZI", 1.0), ("IZZ", 1.0)]

    return [(str(label), float(coeff.real)) for label, coeff in
            zip(cost_hamiltonian.paulis.to_labels(), cost_hamiltonian.coeffs)]


# Number of qubits
n_qubits = num_qubits

# Define cost Hamiltonian
cost_terms = simple_cost_hamiltonian()

# Run ADAPT-QAOA
ansatz, params = adapt_qaoa(n_qubits=n_qubits, cost_terms=cost_terms, max_iters=10, grad_threshold=1e-3)

# Generate final QAOA circuit
final_qc = build_qaoa_circuit(n_qubits, params, ansatz)

# Simulate the final circuit
simulator = Aer.get_backend('statevector_simulator')
result = simulator.run(final_qc).result()
statevector = result.get_statevector()
statevector_array = statevector.data  # 提取 numpy 数组


# Compute expectation value
def compute_expectation(statevector, cost_terms):
    """
    Compute the expectation value of the cost Hamiltonian for the given statevector.
    :param statevector: Simulated quantum state
    :param cost_terms: Cost Hamiltonian terms
    :return: Expectation value
    """
    expectation = 0
    for term, coeff in cost_terms:
        # pauli_op = Pauli(term)
        pauli_op = SparsePauliOp.from_list([(term, coeff)])
        # evolved_op = SparsePauliOp([pauli_op], [coeff])
        # expectation += coeff * np.vdot(statevector, evolved_op.to_matrix() @ statevector)
        expectation += coeff * statevector.expectation_value(pauli_op)
    return np.real(expectation)


# 计算最终能量
final_energy = compute_expectation(statevector, cost_terms)

print("ADAPT-QAOA Simple Problem Results:")
print(f"Selected Ansatz: {ansatz}")
print(f"Optimal Parameters: {params}")
print(f"Final Energy: {final_energy}")


mixer_pool: [[Pauli('XIIIIIIII'), Pauli('IXIIIIIII'), Pauli('IIXIIIIII'), Pauli('IIIXIIIII'), Pauli('IIIIXIIII'), Pauli('IIIIIXIII'), Pauli('IIIIIIXII'), Pauli('IIIIIIIXI'), Pauli('IIIIIIIIX')], [Pauli('YIIIIIIII'), Pauli('IYIIIIIII'), Pauli('IIYIIIIII'), Pauli('IIIYIIIII'), Pauli('IIIIYIIII'), Pauli('IIIIIYIII'), Pauli('IIIIIIYII'), Pauli('IIIIIIIYI'), Pauli('IIIIIIIIY')], [Pauli('XXIIIIIII'), Pauli('XIXIIIIII'), Pauli('XIIXIIIII'), Pauli('XIIIXIIII'), Pauli('XIIIIXIII'), Pauli('XIIIIIXII'), Pauli('XIIIIIIXI'), Pauli('XIIIIIIIX'), Pauli('IXXIIIIII'), Pauli('IXIXIIIII'), Pauli('IXIIXIIII'), Pauli('IXIIIXIII'), Pauli('IXIIIIXII'), Pauli('IXIIIIIXI'), Pauli('IXIIIIIIX'), Pauli('IIXXIIIII'), Pauli('IIXIXIIII'), Pauli('IIXIIXIII'), Pauli('IIXIIIXII'), Pauli('IIXIIIIXI'), Pauli('IIXIIIIIX'), Pauli('IIIXXIIII'), Pauli('IIIXIXIII'), Pauli('IIIXIIXII'), Pauli('IIIXIIIXI'), Pauli('IIIXIIIIX'), Pauli('IIIIXXIII'), Pauli('IIIIXIXII'), Pauli('IIIIXIIXI'), Pauli('IIIIXIIIX'), Pauli('IIIIIXXII'), Pa